# Secondary standard stars

In [30]:
import astropy
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy import units as u
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry
from astropy.wcs import WCS
import warnings
from astropy.wcs import FITSFixedWarning

warnings.filterwarnings('ignore', category=FITSFixedWarning, message=".*'datfix' made the change.*")

# --- 0. Setup ---
wd = r"C:\Users\friesco\workstation\fr-p\studies\ASTRO716\data_excercise"
os.chdir(wd)
plt.rcParams['figure.figsize'] = [10, 8]

# --- 1. Load Calibration Star Data ---
calibration_stars_df = pd.read_csv('calibration_stars.csv')

# --- 2. Load Image Files ---
phot_files = [f for f in os.listdir(wd) if f.startswith('phot_') and f.endswith('.fits')]
phot_files.sort()

# --- 3. Load and Clean PS1 Catalog ---
try:
    ps1_catalog = pd.read_csv('psdr1_new.tsv', sep='\t')

    # Replace invalid entries with NaN (without chained assignment)
    ps1_catalog.loc[ps1_catalog['RAJ2000'] == '-------', 'RAJ2000'] = np.nan
    ps1_catalog.loc[ps1_catalog['DEJ2000'] == '-------', 'DEJ2000'] = np.nan

    # Convert relevant columns to numeric, coercing errors to NaN
    for col in ['RAJ2000', 'DEJ2000', 'rmag', 'e_rmag', 'gmag']:
        ps1_catalog[col] = pd.to_numeric(ps1_catalog[col], errors='coerce')

    # Drop rows with NaN values in essential columns
    ps1_catalog.dropna(subset=['RAJ2000', 'DEJ2000', 'rmag', 'e_rmag'], inplace=True)

    # Now create SkyCoord object
    ps1_coords = SkyCoord(ra=ps1_catalog['RAJ2000'], dec=ps1_catalog['DEJ2000'], unit="deg")

except Exception as e:
    print(f"Error loading or processing PS1 catalog: {e}")
    ps1_coords = None  # or handle it in another appropriate way

# --- 4. Photometry Functions ---
def phot_counts(img, x, y, ap_rad, in_ann, out_ann):
    ap = CircularAperture((x, y), r=ap_rad)
    ann_ap = CircularAnnulus((x, y), r_in=in_ann, r_out=out_ann)
    phot_tbl = aperture_photometry(img, [ap, ann_ap])

    ann_mask = ann_ap.to_mask(method='center')
    ann_data = ann_mask.multiply(img)
    bkg_vals = ann_data[ann_mask.data > 0]
    mean, med, std = sigma_clipped_stats(bkg_vals, sigma=3.0)
    bkg_med = med
    bkg_sum = bkg_med * ap.area

    tot_counts = phot_tbl['aperture_sum_0'][0]
    return tot_counts, bkg_sum, ap.area

def calc_snr(tot_counts, bkg_counts, ap_area):
    mean_counts = (tot_counts - bkg_counts) / ap_area
    if tot_counts + bkg_counts <= 0:
        print("Warning: Negative counts. Returning SNR = 0")
        return 0.0
    err_counts = np.sqrt(tot_counts + bkg_counts) / ap_area
    return mean_counts / err_counts

def phot_mag(img, x, y, ap_rad, in_ann, out_ann):
    ap = CircularAperture((x, y), r=ap_rad)
    ann_ap = CircularAnnulus((x, y), r_in=in_ann, r_out=out_ann)
    phot_tbl = aperture_photometry(img, [ap, ann_ap])

    ann_mask = ann_ap.to_mask(method='center')
    ann_data = ann_mask.multiply(img)
    bkg_vals = ann_data[ann_mask.data > 0]
    mean, med, std = sigma_clipped_stats(bkg_vals, sigma=3.0)
    bkg_med = med
    bkg_sum = bkg_med * ap.area

    final_sum = phot_tbl['aperture_sum_0'] - bkg_sum
    if final_sum[0] > 0:
        inst_mag = -2.5 * np.log10(final_sum) + 50
        mag_err = 1.0857 * (np.sqrt(phot_tbl['aperture_sum_0'] + bkg_sum) / phot_tbl['aperture_sum_0'])
    else:
        inst_mag = np.nan
        mag_err = np.nan
    return inst_mag, mag_err, bkg_med

# --- 5. Optimal Aperture Calculation ---
nx, ny = 1033, 1336  # Nova coordinates
ap_radii_snr = np.arange(5, 40, 1)
in_ann, out_ann = 25, 35
opt_ap = []

for i, f in enumerate(phot_files):
    path = os.path.join(wd, f)
    with fits.open(path) as hdu:
        img = hdu[0].data
        if img.dtype.byteorder == '>':
            img = img.byteswap().view(img.dtype.newbyteorder('<'))

    print(f"\nCalculating optimal aperture for: {f}")
    snrs = []
    for ap_rad in ap_radii_snr:
        tot_counts, bkg_counts, ap_area = phot_counts(img, nx, ny, ap_rad, in_ann, out_ann)
        if tot_counts + bkg_counts <= 0:
            print(f"Warning: Negative counts in {f}, aperture: {ap_rad}")
            snrs.append(0.0)
        else:
            snr = calc_snr(tot_counts, bkg_counts, ap_area)
            snrs.append(snr)

    opt_ap_rad = ap_radii_snr[np.argmax(snrs)]
    opt_ap.append(opt_ap_rad)
    print(f"  Optimal Aperture: {opt_ap_rad} px")

# --- 6. Select Suitable Secondary Standard Stars ---
# Criteria: 14 <= r <= 19, e_rmag <= 0.1, present in all images (or most)

# --- 7. Match Calibration Stars with PS1 Catalog ---
if ps1_coords is not None:
    # Reset index of calibration_stars_df
    calibration_stars_df.reset_index(drop=True, inplace=True)

    calibration_stars_coords = SkyCoord(ra=calibration_stars_df['RAJ2000'], dec=calibration_stars_df['DEJ2000'], unit="deg")
    idx, d2d, d3d = match_coordinates_sky(calibration_stars_coords, ps1_coords)

   # Filter matched stars based on criteria
    good_matches = (ps1_catalog['rmag'].iloc[idx] >= 14) & \
                   (ps1_catalog['rmag'].iloc[idx] <= 19) & \
                   (ps1_catalog['e_rmag'].iloc[idx] <= 0.1)

    # Get the indices of the matched calibration stars
    matched_calibration_indices = np.where(good_matches)[0]

    matched_ps1_indices = idx[good_matches]
    matched_ps1_stars = ps1_catalog.iloc[matched_ps1_indices]

    # Filter calibration_stars_df using the indices of the matched calibration stars
    matched_calibration_stars = calibration_stars_df.iloc[matched_calibration_indices]

else:
    print("Skipping star matching as ps1_coords was not created due to previous errors.")
    matched_ps1_stars = pd.DataFrame()
    matched_calibration_stars = pd.DataFrame()

# --- 8. Perform Photometry on Nova and Selected PS1 Stars ---
photometry_results = []

for i, f in enumerate(phot_files):
    path = os.path.join(wd, f)
    with fits.open(path) as hdu:
        img = hdu[0].data
        hdr = hdu[0].header
        wcs = WCS(hdr)
        if img.dtype.byteorder == '>':
            img = img.byteswap().view(img.dtype.newbyteorder('<'))

    print(f"\nPerforming photometry on: {f}")
    opt_ap_rad = opt_ap[i]

    # Nova
    inst_mag_nova, mag_err_nova, bkg_med_nova = phot_mag(img, nx, ny, opt_ap_rad, in_ann, out_ann)
    ra_nova, dec_nova = wcs.pixel_to_world_values(nx, ny)

    # Selected PS1 Stars
    for index, ps1_star in matched_ps1_stars.iterrows():
        try:
            x, y = wcs.world_to_pixel(SkyCoord(ps1_star['RAJ2000'], ps1_star['DEJ2000'], unit="deg"))
            inst_mag, mag_err, bkg_med = phot_mag(img, x, y, opt_ap_rad, in_ann, out_ann)

            # Convert PS1 r-band magnitude to Sloan r'
            r_prime = ps1_star['rmag'] - 0.001 + 0.011 * (ps1_star['gmag'] - ps1_star['rmag'])
            r_prime_err = 0.004

            photometry_results.append({
                'image': f,
                'mjd': hdr['MJD'],
                'obj_type': 'nova' if index == -1 else 'ps1_star',
                'id': 'nova' if index == -1 else ps1_star['ID'],
                'ra': ra_nova if index == -1 else ps1_star['RAJ2000'],
                'dec': dec_nova if index == -1 else ps1_star['DEJ2000'],
                'x': nx if index == -1 else x,
                'y': ny if index == -1 else y,
                'opt_ap': opt_ap_rad,
                'inst_mag': inst_mag_nova if index == -1 else inst_mag,
                'mag_err': mag_err_nova if index == -1 else mag_err,
                'bkg_med': bkg_med_nova if index == -1 else bkg_med,
                'r_prime': np.nan if index == -1 else r_prime,
                'r_prime_err': np.nan if index == -1 else r_prime_err,
            })
        except Exception as e:
            print(f"Error processing star {ps1_star['ID']} in image {f}: {e}")

# --- 9. Convert Results to DataFrame ---
photometry_results_df = pd.DataFrame(photometry_results)

# --- 10. Display and Save Results ---
print("\nPhotometry Results:")
print(photometry_results_df)
photometry_results_df.to_csv('photometry_results.csv', index=False)


Calculating optimal aperture for: phot_00.fits
  Optimal Aperture: 25 px

Calculating optimal aperture for: phot_01.fits
  Optimal Aperture: 27 px

Calculating optimal aperture for: phot_02.fits
  Optimal Aperture: 26 px

Calculating optimal aperture for: phot_03.fits
  Optimal Aperture: 26 px

Calculating optimal aperture for: phot_04.fits
  Optimal Aperture: 27 px

Calculating optimal aperture for: phot_05.fits
  Optimal Aperture: 25 px

Calculating optimal aperture for: phot_06.fits
  Optimal Aperture: 26 px

Calculating optimal aperture for: phot_07.fits
  Optimal Aperture: 30 px

Calculating optimal aperture for: phot_08.fits
  Optimal Aperture: 33 px

Calculating optimal aperture for: phot_09.fits
  Optimal Aperture: 29 px

Calculating optimal aperture for: phot_10.fits
  Optimal Aperture: 29 px

Calculating optimal aperture for: phot_11.fits
  Optimal Aperture: 30 px

Calculating optimal aperture for: phot_12.fits
  Optimal Aperture: 30 px

Calculating optimal aperture for: pho